In [1]:
from data.fetch_data import *
from strategies.alphas import *
from utils.ga_utils import *
import warnings
from utils.optimizer import *
from utils.data_utils import * 
from utils.train_rf import *


warnings.filterwarnings("ignore")

train_data = fetch_stock_data("BPCL.NS",'2022-01-01','2023-08-08')
test_data= yf.download("BPCL.NS", start = '2023-08-01',end='2024-02-06')



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
def evaluate_strategy(data,alpha_fun,*args):
    data = data.copy()  # Make a copy of the data to avoid modifying the original DataFrame
    data['alpha'] = alpha_fun(data, *args[:-2])  # Assuming you still want to compute alpha
    data['alpha102'] = alpha102(data,2,10)
    stop_loss_percentage = args[-1]
    # print(f"paramters passed to the random forest{args[:-2]}")
    rf_model= train_random_forest(data,test_data,alpha_fun,*args[:-2])


    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    winning_trades = 0
    stricly_winning_trades = 0
    allocated_funds = 100000
    returns = []

    for i in range(1, len(data)):
        try:
            if position == 0:
                # Predict using the random forest model
                if rf_model.predict([[data.iloc[i]['alpha'],data.iloc[i]['alpha102']]])[0] == 1:  # If model predicts 1, enter long position
                    position = 1
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price - (entry_price * 0.01 * stop_loss_percentage)
                    print(f"Entry: {entry_price} on {data.index[i]} Stop Loss: {stop_loss_price}")

                    total_trades += 1
                    no_of_shares = allocated_funds // entry_price
            elif position == 1:
                if data['Close'][i] > entry_price:  # Adjust stop loss only when the price goes up
                    stop_loss_price = max(stop_loss_price, data['Close'][i] - (data['Close'][i] * 0.01 * stop_loss_percentage))
                    
                if data['Low'][i] < stop_loss_price :#:or data['alpha'][i] < threshold_exit:  # add condition for time based exit here
                    position = 0  # Sell due to stop loss
                    exit_price = data['Open'][i] if stop_loss_price > data['Open'][i] else stop_loss_price
                    no_of_shares = allocated_funds//entry_price
                    trade_profit = (exit_price - entry_price) * no_of_shares  # Calculate profit/loss
                    profit=trade_profit
                    allocated_funds += profit
                    return_on_trade = (exit_price -entry_price) * 100/entry_price
                    returns.append(return_on_trade)
                    print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss Allocated Funds {allocated_funds}")
           
                    if(exit_price>= entry_price):
                        winning_trades += 1
                    if(exit_price> entry_price):
                        stricly_winning_trades += 1
            # Rest of the code remains unchanged

        except Exception as ex:
            print("Error occurred:", ex)
            break

    # Calculate and return metrics
    win_rate = winning_trades / total_trades if total_trades != 0 else 0
    metrics = {
        "Profit": profit,
        "Total Trades": total_trades,
        "Winning Trades": winning_trades,
        "Strictly Winning Trades": stricly_winning_trades,
        "Win Rate": win_rate,
        "Allocated Funds": allocated_funds,
        "Sharpe Ratio": calculate_sortino_ratio(pd.Series(returns))
    }
    return metrics,rf_model


In [3]:
def evaluate_strategy_test(data,alpha_fun,rf_model,*args):
    data = data.copy()  # Make a copy of the data to avoid modifying the original DataFrame
    data['alpha'] = alpha_fun(data, *args[:-2])  # Assuming you still want to compute alpha
    data['alpha102'] = alpha102(data,2,10)
    stop_loss_percentage = args[-1]
    # print(f"paramters passed to the random forest{args[:-2]}")


    position = 0  # 0 for no position, 1 for long position
    profit = 0
    total_trades = 0
    winning_trades = 0
    stricly_winning_trades = 0
    allocated_funds = 100000
    returns = []



    for i in range(1, len(data)):
        try:
            if position == 0:
                # Predict using the random forest model
                if rf_model.predict([[data.iloc[i]['alpha'],data.iloc[i]['alpha102']]])[0] == 1:  # If model predicts 1, enter long position
                    position = 1
                    entry_price = data['Close'][i]
                    stop_loss_price = entry_price - (entry_price * 0.01 * stop_loss_percentage)
                    print(f"Entry: {entry_price} on {data.index[i]} Stop Loss: {stop_loss_price}")

                    total_trades += 1
                    no_of_shares = allocated_funds // entry_price
            elif position == 1:
                if data['Close'][i] > entry_price:  # Adjust stop loss only when the price goes up
                    stop_loss_price = max(stop_loss_price, data['Close'][i] - (data['Close'][i] * 0.01 * stop_loss_percentage))
                    
                if data['Low'][i] < stop_loss_price :#:or data['alpha'][i] < threshold_exit:  # add condition for time based exit here
                    position = 0  # Sell due to stop loss
                    exit_price = data['Open'][i] if stop_loss_price > data['Open'][i] else stop_loss_price
                    no_of_shares = allocated_funds//entry_price
                    trade_profit = (exit_price - entry_price) * no_of_shares  # Calculate profit/loss
                    profit=trade_profit
                    allocated_funds += profit
                    return_on_trade = (exit_price -entry_price) * 100/entry_price
                    returns.append(return_on_trade)
                    print(f"Exit: {exit_price} on {data.index[i]} exited on stop loss Allocated Funds {allocated_funds}")
           
                    if(exit_price>= entry_price):
                        winning_trades += 1
                    if(exit_price> entry_price):
                        stricly_winning_trades += 1
            # Rest of the code remains unchanged

        except Exception as ex:
            print("Error occurred:", ex)
            break

    # Calculate and return metrics
    win_rate = winning_trades / total_trades if total_trades != 0 else 0
    metrics = {
        "Profit": profit,
        "Total Trades": total_trades,
        "Winning Trades": winning_trades,
        "Strictly Winning Trades": stricly_winning_trades,
        "Win Rate": win_rate,
        "Allocated Funds": allocated_funds,
        "Sharpe Ratio": calculate_sortino_ratio(pd.Series(returns))
    }
    
    return metrics





In [4]:
met,rf = evaluate_strategy(train_data,alpha12,9,0,1)

this is being trained upon                alpha   alpha102
Date                           
2022-01-03       NaN   0.000000
2022-01-04  0.149994  -0.072724
2022-01-05 -8.750000   4.158681
2022-01-06  1.099976   5.341940
2022-01-07 -0.400024   5.211090
...              ...        ...
2023-08-01  0.000000  -3.547411
2023-08-02  8.399994  -6.909129
2023-08-03 -1.100006  -7.521828
2023-08-04  7.600006 -10.462042
2023-08-07  0.250000  -9.874580

[396 rows x 2 columns]
Accuracy: 0.4566929133858268
Entry: 385.95001220703125 on 2022-01-04 00:00:00 Stop Loss: 382.09051208496095
Exit: 384.95001220703125 on 2022-01-05 00:00:00 exited on stop loss Allocated Funds 99741.0
Entry: 391.1499938964844 on 2022-01-11 00:00:00 Stop Loss: 387.23849395751955
Exit: 387.23849395751955 on 2022-01-12 00:00:00 exited on stop loss Allocated Funds 98747.47901550293
Entry: 381.70001220703125 on 2022-01-21 00:00:00 Stop Loss: 377.8830120849609
Exit: 377.8830120849609 on 2022-01-24 00:00:00 exited on stop loss Allocate

In [8]:
a =evaluate_strategy_test(test_data,alpha12,rf,9,0,1)
a

Entry: 357.8999938964844 on 2023-08-16 00:00:00 Stop Loss: 354.32099395751953
Exit: 354.32099395751953 on 2023-08-18 00:00:00 exited on stop loss Allocated Funds 99001.45901702881
Entry: 352.04998779296875 on 2023-08-21 00:00:00 Stop Loss: 348.52948791503906
Exit: 348.52948791503906 on 2023-08-22 00:00:00 exited on stop loss Allocated Funds 98012.19855133057
Entry: 344.79998779296875 on 2023-08-23 00:00:00 Stop Loss: 341.35198791503905
Exit: 347.45001220703125 on 2023-08-24 00:00:00 exited on stop loss Allocated Funds 98764.80548492432
Entry: 356.79998779296875 on 2023-08-29 00:00:00 Stop Loss: 353.23198791503904
Exit: 353.20001220703125 on 2023-08-30 00:00:00 exited on stop loss Allocated Funds 97771.21222320557
Entry: 340.54998779296875 on 2023-08-31 00:00:00 Stop Loss: 337.14448791503906
Exit: 340.7580120849609 on 2023-09-01 00:00:00 exited on stop loss Allocated Funds 97830.91519500731
Entry: 344.70001220703125 on 2023-09-04 00:00:00 Stop Loss: 341.2530120849609
Exit: 345.899993896

{'Profit': 814.8023681640625,
 'Total Trades': 24,
 'Winning Trades': 13,
 'Strictly Winning Trades': 12,
 'Win Rate': 0.5416666666666666,
 'Allocated Funds': 96685.47396514892,
 'Sharpe Ratio': -0.24961556240348376}

In [6]:
# run_ga_optimization(train_data,generate_individual,alpha14,evaluate_strategy)
print(a)

{'Profit': 814.8023681640625, 'Total Trades': 24, 'Winning Trades': 13, 'Strictly Winning Trades': 12, 'Win Rate': 0.5416666666666666, 'Allocated Funds': 96685.47396514892, 'Sharpe Ratio': -0.24961556240348376}


In [7]:
met

{'Profit': -416.5,
 'Total Trades': 78,
 'Winning Trades': 37,
 'Strictly Winning Trades': 34,
 'Win Rate': 0.47435897435897434,
 'Allocated Funds': 90811.18354919435,
 'Sharpe Ratio': -0.2643040873788455}